In [1]:
import faiss
from representations.contracts import contracts
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')

In [3]:
from tqdm import tqdm 
for contract, paragraph_gen in tqdm(contracts.items()):
    paragraphs = list(paragraph_gen())
        

100%|██████████| 7400/7400 [01:32<00:00, 79.69it/s] 
